In [4]:
import os , json , time , sqlite3, warnings
from typing import Dict, Any , List
import re
import easyocr
# from euriapi.langgraph import EuriaiLangGraph



In [5]:
EURI_API_KEY ="euri-260e447ef9308db43bf0f5259e1913fdeca5592c058784c3e46bba5d9dd45f39"
MODEL = "gpt-4.1-nano"


In [6]:
INPUT_DIR = "invoices"
DB_PATH = "invoices.sqllite"
PROCESSED_DIR = "processed.json"
POLL_SEC = 5

In [7]:
os.makedirs(INPUT_DIR,  exist_ok=True)


In [9]:
NODES = ["WATCH", "OCR", "CLEAN" , "EXTRACT", "VALIDATE", "PERSIST" , "NOTIFY"]
EDGES = [("WATCH", "OCR"),
    ("OCR", "CLEAN"),
    ("CLEAN", "EXTRACT"),
    ("EXTRACT", "VALIDATE"),
    ("VALIDATE", "PERSIST"),
    ("PERSIST", "NOTIFY")
     ]

In [11]:
def load_seen() -> set:
    if not os.path.exists(PROCESSED_DIR):
        return set()
try:

    with open(PROCESSED_DIR, "r", encoding="utf-8") as f:
        return set(json.load(f))
    return set()
except Exception as e:
    warnings.warn(f"Could not load seen file {PROCESSED_DIR} : {e}")
    return set()    

SyntaxError: 'return' outside function (1272888063.py, line 7)

In [ ]:
def save_seen(seen: set):
    with open(PROCESSED_DIR, "w", encoding="utf-8") as f:
        json.dump(list(seen), f)

In [ ]:
def ensure_schema():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS invoices (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        file_name TEXT,
        vendor TEXT,
        numer TEXT,
        date TEXT,
        total REAL
        currency TEXT.
        raw_json TEXT,
    )
    """)
# add columns if table existed with an older schema
    cursor.execute("PRAGMA table_info(invoices)")
cols = {row[1] for row in cursor.fetchall()}
if file_name not in cols:
    cursor.execute("ALTER TABLE invoices ADD COLUMN file_name TEXT")
if raw_json not in cols:
    cursor.execute("ALTER TABLE invoices ADD COLUMN raw_json TEXT")



    conn.commit()
    conn.close()

In [ ]:
ocr = easyocr.Reader(["en"], gpu=False)


In [ ]:
clean_graph = EuriaiLangGraph(
    api_key=EURI_API_KEY, default_model=MODEL)
    clean_graph.add_ai_node(
        "CLEAN",
        
        description="Cleans the OCR text to make it easier to extract information from. Remove line breaks, hyphenation, extra spaces, etc.",
    )
    extract_graph.set_entry_point("CLEAN")
    clean_graph.add_edges([("OCR", "CLEAN")])  
    
extract_graph = EuriaiLangGraph(
    api_key=EURI_API_KEY, default_model=MODEL) 